Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
EC = str(Path('~/proj/encoded_client').expanduser())
if EC not in sys.path:
    sys.path.append(EC)

from encoded_client.encoded import ENCODED, Document, typed_column_parser, DCCValidator

In [3]:
HTSW = str(Path('~/proj/htsworkflow').expanduser())
if HTSW not in sys.path:
    sys.path.append(HTSW)

In [4]:

from htsworkflow.submission.aws_submission import run_aws_cp
from htsworkflow.util.api import (
    add_auth_options,
    make_auth_from_opts,
    HtswApi,
)

In [5]:
config = configparser.ConfigParser()
config.read([os.path.expanduser('~/.htsworkflow.ini'),
             '/etc/htsworkflow.ini'
             ])

SECTION = 'sequence_archive'
if config.has_section(SECTION):
    apiid = config.get(SECTION, 'apiid')
    apikey = config.get(SECTION, 'apikey')
    apihost = config.get(SECTION, 'host')

apihost='http://jumpgate.caltech.edu'    
auth = {'apiid': apiid, 'apikey': apikey }
htsw = HtswApi(apihost, auth)

In [6]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-23369-23384-mats-cells.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [7]:
award = 'UM1HG009443'

# Retrieve insert size

In [8]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
for i, row in libraries.iterrows():
    accession = row['accession']
    alias = row['aliases:array']
    fragment = row['average_fragment_size:integer']
    library_id = alias[len('barbara-wold:'):]
    jumpgate_info = htsw.get_library(library_id)
    print(library_id, fragment, jumpgate_info['insert_size'])
        #if jumpgate_info['insert_size'] != dcc_info.get('average_fragment_size'):
        #    server.patch_json(dcc_info['@id'], {'average_fragment_size': int(jumpgate_info['insert_size'])})

/home/diane/woldlab/ENCODE/stranded-23369-23384-mats-cells.xlsx
23369 254 254
23370 228 228
23371 257 257
23372 252 252
23373 260 260
23374 258 258
23375 259 259
23376 255 255
23380 252 252
23381 257 257
23382 251 251
23383 255 255
23384 245 245
23377 247 247
23378 254 254
23379 249 249


# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [9]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
        biosample.loc[i, 'source'] = obj['source']['@id']
        biosample.loc[i, 'uuid'] = obj['uuid']
        donor = obj['donor']
        if isinstance(donor, dict):
            biosample.loc[i, 'donor'] = donor['@id']
            
biosample

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,description,library_id:skip,cDNA_sample:skip,organism,source,lab,award,donor,nih_institutional_certification
0,78507c69-c849-4755-b531-84719ee39693,ENCBS511ZVG,barbara-wold:ENC4_cDNA613,/biosample-types/cell_line_EFO_0002847/,PC-9,PC-9 rep1_ENC4_cDNA613,23369,ENC4_cDNA613,/organisms/human/,/sources/sigma/,barbara-wold,UM1HG009443,/human-donors/ENCDO647UHQ/,NIC00015
1,34c5f71f-241e-4b09-93a7-236a14042f91,ENCBS021NLC,barbara-wold:ENC4_cDNA614,/biosample-types/cell_line_EFO_0002847/,PC-9,PC-9 rep2_ENC4_cDNA614,23370,ENC4_cDNA614,/organisms/human/,/sources/sigma/,barbara-wold,UM1HG009443,/human-donors/ENCDO647UHQ/,NIC00015
2,0455c126-3abe-430e-9657-36dad0cd03b6,ENCBS869GJQ,barbara-wold:ENC4_cDNA615,/biosample-types/cell_line_EFO_0006711/,OCI-LY7,OCI-LY7 rep1_ENC4_cDNA615,23371,ENC4_cDNA615,/organisms/human/,/sources/dsmz/,barbara-wold,UM1HG009443,/human-donors/ENCDO351AAA/,NaN
3,2824c1c7-9407-45fe-b5e6-dc627c54cc76,ENCBS112SEO,barbara-wold:ENC4_cDNA616,/biosample-types/cell_line_EFO_0006711/,OCI-LY7,OCI-LY7 rep2_ENC4_cDNA616,23372,ENC4_cDNA616,/organisms/human/,/sources/dsmz/,barbara-wold,UM1HG009443,/human-donors/ENCDO351AAA/,NaN
4,e0246145-739f-4f55-83e8-c46f784c96b3,ENCBS820SXP,barbara-wold:ENC4_cDNA617,/biosample-types/primary_cell_CL_0002618/,endothelial cell of umbilical vein,HUVEC rep1_ENC4_cDNA617,23373,ENC4_cDNA617,/organisms/human/,/sources/lonza/,barbara-wold,UM1HG009443,/human-donors/ENCDO000AAS/,NIC00044
5,7fd295b5-2947-481b-a479-e503e656eb3b,ENCBS023JCN,barbara-wold:ENC4_cDNA618,/biosample-types/primary_cell_CL_0002618/,endothelial cell of umbilical vein,HUVEC rep2_ENC4_cDNA618,23374,ENC4_cDNA618,/organisms/human/,/sources/lonza/,barbara-wold,UM1HG009443,/human-donors/ENCDO000AAS/,NIC00044
6,a042d3f0-8b14-4ac2-9b22-298b4e9eb9d1,ENCBS996RLF,barbara-wold:ENC4_cDNA619,/biosample-types/primary_cell_CL_0002327/,mammary epithelial cell,HMEC rep1_ENC4_cDNA619,23375,ENC4_cDNA619,/organisms/human/,/sources/lonza/,barbara-wold,UM1HG009443,/human-donors/ENCDO343AAA/,NIC00044
7,9c292e86-c537-4049-97ec-d377561cdea5,ENCBS710VQU,barbara-wold:ENC4_cDNA620,/biosample-types/primary_cell_CL_0002327/,mammary epithelial cell,HMEC rep2_ENC4_cDNA620,23376,ENC4_cDNA620,/organisms/human/,/sources/lonza/,barbara-wold,UM1HG009443,/human-donors/ENCDO343AAA/,NIC00044
8,729c34d3-488d-4a42-a7e0-b01e1d6fe9c8,ENCBS020DQC,barbara-wold:ENC4_cDNA624,/biosample-types/cell_line_EFO_0001099/,Caco-2,Caco-2 rep2_ENC4_cDNA624,23380,ENC4_cDNA624,/organisms/human/,/sources/atcc/,barbara-wold,UM1HG009443,/human-donors/ENCDO000ACR/,NaN
9,5e7982c6-cffe-473d-86fb-42fd8df97aee,ENCBS961BBX,barbara-wold:ENC4_cDNA625,/biosample-types/cell_line_EFO_0002819/,Calu3,Calu3 rep1_ENC4_cDNA625,23381,ENC4_cDNA625,/organisms/human/,/sources/atcc/,barbara-wold,UM1HG009443,/human-donors/ENCDO109DGE/,NaN


In [10]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Biosamples

In [11]:
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosamples, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [12]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Update biosample aliases

In [13]:
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
for i, row in biosamples.iterrows():
    obj = server.get_json('/biosamples/{}/'.format(row.accession))
    new_aliases = obj['aliases'].copy()
    sheet_aliases = typed_column_parser('aliases:array', row['aliases:array'])[1]
    for alias in sheet_aliases:
        if alias not in obj['aliases']:
            new_aliases.append(alias)
    if len(new_aliases) > len(sheet_aliases):
        print("Would update: {} {} -> {}".format(row.accession, row['aliases:array'], new_aliases))

Would update: ENCBS511ZVG barbara-wold:ENC4_cDNA613 -> ['mats-ljungman:PC-9_4001_BW1', 'barbara-wold:ENC4_cDNA613']
Would update: ENCBS021NLC barbara-wold:ENC4_cDNA614 -> ['mats-ljungman:PC-9_4002_BW1', 'barbara-wold:ENC4_cDNA614']
Would update: ENCBS869GJQ barbara-wold:ENC4_cDNA615 -> ['mats-ljungman:OCI-LY7_4001_BW1', 'barbara-wold:ENC4_cDNA615']
Would update: ENCBS112SEO barbara-wold:ENC4_cDNA616 -> ['mats-ljungman:OCI-LY7_4002_BW1', 'barbara-wold:ENC4_cDNA616']
Would update: ENCBS820SXP barbara-wold:ENC4_cDNA617 -> ['mats-ljungman:HUVEC_4001_BW1', 'barbara-wold:ENC4_cDNA617']
Would update: ENCBS023JCN barbara-wold:ENC4_cDNA618 -> ['mats-ljungman:HUVEC_4002_BW1', 'barbara-wold:ENC4_cDNA618']
Would update: ENCBS996RLF barbara-wold:ENC4_cDNA619 -> ['mats-ljungman:HMEC_4001_BW1', 'barbara-wold:ENC4_cDNA619']
Would update: ENCBS710VQU barbara-wold:ENC4_cDNA620 -> ['mats-ljungman:HMEC_4002_BW1', 'barbara-wold:ENC4_cDNA620']
Would update: ENCBS020DQC barbara-wold:ENC4_cDNA624 -> ['mats-lj

In [14]:
new_aliases, sheet_aliases

(['mats-ljungman:Caco-2_4001_BW1', 'barbara-wold:ENC4_cDNA623'],
 ['barbara-wold:ENC4_cDNA623'])

# Register Libraries

In [27]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/',
                            libraries,
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/stranded-23369-23384-mats-cells.xlsx
0


In [26]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [31]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [30]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [35]:
print(server.server)
print(spreadsheet_name)
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
/home/diane/woldlab/ENCODE/stranded-23369-23384-mats-cells.xlsx
0


In [34]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [21]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))

16
